In [1]:
!pip install sqlalchemy==1.3.9 ipython-sql prettytable pandas

In [3]:
import pkg_resources

required = {
    'sqlalchemy': '1.3.9',
    'ipython-sql': '',
    'prettytable': '',
    'pandas': ''
}

for package, version in required.items():
    try:
        installed = pkg_resources.get_distribution(package).version
        if version and installed != version:
            print(f"⚠️ {package} version mismatch (installed: {installed}, required: {version})")
        else:
            print(f"✅ {package} ({installed}) is installed")
    except pkg_resources.DistributionNotFound:
        print(f"❌ {package} is NOT installed")

✅ sqlalchemy (1.3.9) is installed
✅ ipython-sql (0.4.1) is installed
✅ prettytable (0.7.2) is installed
✅ pandas (2.2.2) is installed


In [5]:
!pip install --force-reinstall sqlalchemy==1.3.9

  Using cached SQLAlchemy-1.3.9-cp312-cp312-win_amd64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aext-project-filebrowser-server 4.1.0 requires sqlalchemy>=2.0.29, but you have sqlalchemy 1.3.9 which is incompatible.


In [5]:
import pkg_resources

required = {
    'sqlalchemy': '1.3.9',  # Must be exact
    'ipython-sql': '',       # Any version
    'prettytable': '',       # Any version
    'pandas': ''            # Any version
}

print("Package Verification Results:")
for package, version in required.items():
    try:
        installed = pkg_resources.get_distribution(package).version
        if version:
            if installed == version:
                print(f"✅ {package} ({installed}) - CORRECT VERSION")
            else:
                print(f"⚠️ {package} ({installed}) - REQUIRES VERSION {version}")
        else:
            print(f"✅ {package} ({installed}) - INSTALLED")
    except pkg_resources.DistributionNotFound:
        print(f"❌ {package} - NOT INSTALLED")

Package Verification Results:
✅ sqlalchemy (1.3.9) - CORRECT VERSION
✅ ipython-sql (0.4.1) - INSTALLED
✅ prettytable (0.7.2) - INSTALLED
✅ pandas (2.2.2) - INSTALLED


In [7]:
# Test SQLAlchemy
import sqlalchemy
print(f"SQLAlchemy version: {sqlalchemy.__version__}")  # Should show 1.3.9

# Test SQL magic
%load_ext sql
print("SQL extension loaded successfully!")

# Test table rendering
from prettytable import PrettyTable
pt = PrettyTable()
pt.add_column("Test", ["✅ Working"])
print(pt)

SQLAlchemy version: 1.3.9
SQL extension loaded successfully!
+-----------+
|    Test   |
+-----------+
| ✅ Working |
+-----------+


In [30]:
%load_ext sql
import sqlite3
import pandas as pd

# Create raw connection
conn = sqlite3.connect('spacex.db')

# Load data
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql('SPACEXTBL', conn, if_exists='replace', index=False)

# Create cleaned table
conn.execute("DROP TABLE IF EXISTS SPACEXTABLE")
conn.execute("CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL")
conn.commit()
conn.close()

# Connect with SQL magic
%sql sqlite:///spacex.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [32]:
%%sql
SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE;

 * sqlite:///spacex.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [34]:
%%sql
SELECT * FROM SPACEXTABLE 
WHERE "Launch_Site" LIKE 'CCA%' 
LIMIT 5;

 * sqlite:///spacex.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [36]:
%%sql
SELECT SUM("Payload_Mass__kg_") 
FROM SPACEXTABLE 
WHERE "Customer" = 'NASA (CRS)';

 * sqlite:///spacex.db
Done.


"SUM(""Payload_Mass__kg_"")"
45596


In [38]:
%%sql
SELECT AVG("Payload_Mass__kg_") 
FROM SPACEXTABLE 
WHERE "Booster_Version" = 'F9 v1.1';

 * sqlite:///spacex.db
Done.


"AVG(""Payload_Mass__kg_"")"
2928.4


In [40]:
%%sql
SELECT MIN("Date") 
FROM SPACEXTABLE 
WHERE "Landing_Outcome" = 'Success (ground pad)';

 * sqlite:///spacex.db
Done.


"MIN(""Date"")"
2015-12-22


In [42]:
%%sql
SELECT DISTINCT "Booster_Version" 
FROM SPACEXTABLE 
WHERE "Landing_Outcome" LIKE 'Success (drone ship)%'
AND "Payload_Mass__kg_" BETWEEN 4000 AND 6000;

 * sqlite:///spacex.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [44]:
%%sql
SELECT 
    CASE 
        WHEN "Landing_Outcome" LIKE 'Success%' THEN 'Success'
        ELSE 'Failure'
    END AS Outcome,
    COUNT(*) AS Count
FROM SPACEXTABLE
GROUP BY Outcome;

 * sqlite:///spacex.db
Done.


Outcome,Count
Failure,40
Success,61


In [46]:
%%sql
SELECT "Booster_Version" 
FROM SPACEXTABLE
WHERE "Payload_Mass__kg_" = (
    SELECT MAX("Payload_Mass__kg_") 
    FROM SPACEXTABLE
);

 * sqlite:///spacex.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [48]:
%%sql
SELECT 
    substr(Date, 6, 2) AS Month,
    "Booster_Version",
    "Launch_Site"
FROM SPACEXTABLE
WHERE "Landing_Outcome" LIKE 'Failure (drone ship)%'
AND substr(Date, 0, 5) = '2015';

 * sqlite:///spacex.db
Done.


Month,Booster_Version,Launch_Site
01,F9 v1.1 B1012,CCAFS LC-40
04,F9 v1.1 B1015,CCAFS LC-40


In [50]:
%%sql
SELECT 
    "Landing_Outcome",
    COUNT(*) AS Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY Count DESC;

 * sqlite:///spacex.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
